## Введение в глубокое обучение
### Занятие 6. Автоэнкодеры и векторные представления слов.

##### Автор: [Радослав Нейчев](https://www.linkedin.com/in/radoslav-neychev/), @neychev
##### Credits: embeddings visualization is based on the notebook by [YSDA NLP course](https://github.com/yandexdataschool/nlp_course)

#### План занятия:
0. Повторение основных моментов при работе с текстами
1. Автоэнкодеры, снижение размерности и построение информативных представлений данных
2. Построение информативных представлений слов. Обзор word2vec
3. Работа с предобученными векторными представлениями слов

### 0. Повторение основных моментов при работе с текстами
Вспомним основные отличия текста от табличных данных:
1. Текст представляет собой последовательность токенов из конечного алфавита, т.е. все элементы последовательности принимают дискретные значения.
2. Текст может быть _переменной длины_.

__Токен__ – минимальный и неделимый элемент текстовой последовательности. В зависимости от выбора эксперта, в качестве токенов могут выступать как символы, так и морфемы, слова или даже группы слов.

### 1. Автоэнкодеры, снижение размерности и построение информативных представлений данных
__Go to slides__

### 2. Построение информативных векторных представлений слов. Обзор word2vec.
Человек воспринимает слова знакомого языка как отдельные смысловые единицы. Затем из них складывается смысл всего предложения или текста. Машина, в отличие от человека, не имеет представления о смысле слов, она не владеет языком. Для машины слова являются лишь последовательностью некоторых символов. Чтобы использовать всю доступную информацию при работе с текстом необходимы информативные представления слов. Уже рассмотренное one-hot кодирование имеет несколько минусов.

##### __One-hot кодирование__
Каждому токену в словаре был сопоставлен уникальный индекс. Тогда токену можно поставить в соответствие вектор размера словаря, где единственное ненулевое значение стоит на соответствующей токену позиции. Например, слово "самолет" сопоставлено индексу 0, а размерность словаря 5. Данному токену соответствует вектор `[1, 0, 0, 0, 0]`.
Слово "обед" сопоставлено индексу 4, поэтому ему соответствует вектор `[0, 0, 0, 0, 1]`.

Получается, что векторные представления всех слов ортогональны друг другу, и слово "кот" удалено от слова "самолет" так же, как и от слова "кошка". Для владеющего языком человека разница между словами очевидна, как и то, что "кошка" и "кот" гораздо ближе друг к другу по смыслу, чем "кошка" и "самолет". При построении информативного векторного представления также хотелось бы установить смысловую близость слов. Учесть связи между словами и их смысл нам поможет word2vec.

#### __Построение эмбеддингов с помощью word2vec__.

__Go to slides__


С этим может помочь простая мысль (озвученная в различных формах множество раз): __слово в значительной мере определяется контекстом, в котором оно встречается__. На основании чего можно сделать простой вывод: для некоторых слов более характерен один контекст, а для других – другой. Именно на этой идее и построен word2vec (как и многие другие эмбеддинги).

По слову можно научиться предсказывать контекст, в котором оно встречается. Конечно, результат не будет идеально точным. Но если модель делает предсказания лучше, чем случайным образом, значит, она улавливает какую-то связь. И тогда внутреннее представление модели для каждого слова и может использоваться в качестве искомого векторного представления, причем и в других задачах.


Формулировка гипотезы выше (слово значительно связано с контекстом) позволяет использовать в качестве обучающей выборки все множество текстов для выбранного языка. Собрание сочинений классиков, статьи в энциклопедии, новостные заметки – все это становится обучающей выборкой. И векторные представления, полученные на основе данных текстов позволяют улавливать связь между этими словами.

In [ ]:
! pip install --upgrade nltk gensim bokeh umap-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.4 MB/s 
     |████████████████████████████████| 18.5 MB 1.2 MB/s 
     |████████████████████████████████| 88 kB 7.6 MB/s 
     |████████████████████████████████| 1.1 MB 53.4 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82830 sha256=a03cb4644d8c48cedf6d457b317541edbcccd4d7e9177901b2a2ab016fda5a9c
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.8-py3-none-any.whl size=55512 sha256=8859139402ed6210f8e6055b4a00a730d781f30e1429a34926a9d51c2fbf36e6
  Stored in directory: /root/.cache/pip/wheels/19/bc/eb/974072a56a7082a302f8b4be1ad6d21bf5019235c2eff65928
Successfully built umap-learn pynndescent
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gen

In [ ]:
import itertools
import string

import numpy as np
import umap
from nltk.tokenize import WordPunctTokenizer

from matplotlib import pyplot as plt

from IPython.display import clear_output

In [ ]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt -nc
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

--2022-11-09 12:47:40--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/obaitrix9jyu84r/quora.txt [following]
--2022-11-09 12:47:41--  https://www.dropbox.com/s/dl/obaitrix9jyu84r/quora.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc01ccd6b4729f72a1877d87a92d.dl.dropboxusercontent.com/cd/0/get/BwbV2OZqyRMo7OVEBtnI2PME3T7HHfYTnxaARvRPGe75AgAnkvHP_xQBEX6TkmwYq6FI11Vg-nfC1SZbx_e2XQPPT8POvekFiDxzeBgAcNu82xAVlhzpNLEUEUQUbOwd38EDArA8_SSwstLeM6VFkALYHrBSp6BlJ-lmJRqUenMXwA/file?dl=1# [following]
--2022-11-09 12:47:41--  https://uc01ccd6b4729f72a1877d87a92d.dl.dropboxusercontent.com/cd/0/get/BwbV2OZqyRMo7OVEBtnI2PME3T7HHfYTnxaARvRPGe75AgAnkvHP_xQBEX6TkmwYq6FI11Vg-nfC1SZbx_e2XQP

In [ ]:
data = list(open("./quora.txt", encoding="utf-8"))
data[55]

'What are all the pros and cons of having dual citizenship?\n'

Произведем токенизацию и базовую предобработку:

In [ ]:
tokenizer = WordPunctTokenizer()
print(tokenizer.tokenize(data[50]))
data_tok = [tokenizer.tokenize(x.lower()) for x in data]

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [ ]:
len(data)

537272

Для начала, обучим word2vec на доступном наборе данных. Строить для этого модель вручную не понадобится, она уже доступна в `gensim`.

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok,
                 vector_size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

Теперь нам доступны векторы для любого слова из словаря:

In [ ]:
# now you can get word vectors !
model.get_vector('cat')

array([-1.6260289 ,  0.72172344, -0.22443022,  0.02166116,  0.6309772 ,
        1.8932906 , -0.2702431 , -1.5375609 , -0.46870956,  3.3435225 ,
        1.9519538 , -1.5382302 ,  0.7857858 ,  1.905574  ,  1.4623067 ,
        0.37732717, -5.0946913 , -1.4828953 , -1.4123696 , -2.514107  ,
        2.3614693 ,  0.47887847, -2.9216616 ,  0.1493887 ,  0.5817923 ,
        0.27198172, -1.4305896 , -3.257401  , -3.4542992 ,  0.54872507,
       -0.6800773 ,  0.00997528], dtype=float32)

Так как слова представлены векторами, теперь можно вычислить расстояние (или некоторую меру схожести) между ними. Например, можно оценить, какие слова наиболее близки к заданному.

In [ ]:
# or query similar words directly. Go play with it!
model.most_similar('pet')

[('dog', 0.7703542709350586),
 ('parent', 0.755887508392334),
 ('cake', 0.7409449815750122),
 ('baby', 0.736035943031311),
 ('teenager', 0.7299429178237915),
 ('ass', 0.7237233519554138),
 ('alcoholic', 0.7172170281410217),
 ('hobby', 0.7158686518669128),
 ('shower', 0.7030816674232483),
 ('whale', 0.6956217288970947)]

#### Использование предобученных векторных представлений
Для получения качественных эмбеддингов стоит использовать большие наборы данных. Также бывает полезным использовать данные из определенной предметной области. Конечно, обучение занимает значительное время, поэтому зачастую используются предобученные эмбеддинги для слов.

Загрузим предобученные эмбеддинги небольшой размерности (25). Они были обучены на данных из Twitter.

In [ ]:
import gensim.downloader as api
model = api.load('glove-twitter-25')

[==================================================] 100.0% 104.8/104.8MB downloaded


In [ ]:
model.most_similar(positive=["лето"])#, negative=["animal"])

[('утро', 0.8916537761688232),
 ('весна', 0.878777265548706),
 ('зима', 0.8693447113037109),
 ('скоро', 0.8552830219268799),
 ('солнце', 0.8396058082580566),
 ('ночь', 0.8204128742218018),
 ('надеюсь', 0.8200833797454834),
 ('воскресенье', 0.8176007270812988),
 ('летом', 0.8077778816223145),
 ('солнышко', 0.8064454793930054)]

#### Визуализация векторных представлений слов

В данный момент каждое слово представлено вектором размерности 25. Для визуализации слов нам понадобится техника снижения размерности. Для простоты можно воспользоваться методом главных компонент, PCA, который также может рассматриваться как линейный автоэнкодер:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min,$$
где $W$ и $\hat{W}$ – обучаемые параметры.

In [ ]:
model.sort_by_descending_frequency()

In [ ]:
len(model.key_to_index.keys())

1193514

In [ ]:
words = list(model.key_to_index.keys())[:1000]

print(words[::100])

word_vectors = np.asarray([model[x] for x in words])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
pca = PCA(2)
scaler = StandardScaler()

word_vectors_pca = scaler.fit_transform(word_vectors)
word_vectors_pca = pca.fit_transform(word_vectors_pca)

Для визуализации обратимся к замечательной библиотеке `bokeh`. Графики являются интерактивными.

In [ ]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [ ]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

Figure(id='1003', ...)

#### Снижение размерности с помощью UMAP
Метод главных компонент – замечательная техника, но он позволяет улавливать лишь линейные зависимости в данных. Обратимся к технике [UMAP](https://habr.com/ru/company/newprolab/blog/350584/), которая учитывает соседей заданных точек. По ссылке выше можно прочитать развернутое описание данной техники.

In [ ]:
embedding = umap.UMAP(n_neighbors=5).fit_transform(word_vectors)

In [ ]:
draw_vectors(embedding[:, 0], embedding[:, 1], token=words)

Figure(id='1113', ...)

Как можно видеть, слова образуют группы, различимые невооруженным глазом. График - интерактивный, при наведении курсора на точку указывается, какому слову она соответствует.

#### Визуализация фраз

В завершение построим векторные представления не только слов, но и целых фраз. Для простоты воспользуемся подходом, аналогичным мешку слов (BoW): мешком эмбеддингов. Каждую фразу представим в виде усредненного эмбеддинга всех входящих в нее слов.

In [ ]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings.
    """
    vector = np.zeros([model.vector_size], dtype='float32')
    phrase_tokenized = tokenizer.tokenize(phrase.lower())
    phrase_vectors = [model[x] for x in phrase_tokenized if model.has_index_for(x)]

    if len(phrase_vectors) != 0:
        vector = np.mean(phrase_vectors, axis=0)
    return vector

In [ ]:
data[402687]

'What gift should I give to my girlfriend on her birthday?\n'

In [ ]:
get_phrase_embedding(data[402687])

array([-0.18204999,  0.30953574,  0.20861094,  0.07982156, -0.22565515,
       -0.33001748,  1.2495784 ,  0.13134292, -0.33788875,  0.06196944,
       -0.231793  ,  0.09389219, -4.9685497 , -0.23611419, -0.32609668,
       -0.092073  ,  0.4407505 , -0.75413746, -0.5389092 , -0.184752  ,
        0.07867809,  0.20018655, -0.16202375,  0.30375698, -0.41255665],
      dtype=float32)

In [ ]:
vector = get_phrase_embedding("Hello, today we speak about word vectors!")

In [ ]:
vector

array([ 0.30748555, -0.05400768, -0.18464424, -0.13607563, -0.23507245,
       -0.04939345,  1.1183112 ,  0.05371887, -0.32848334,  0.19103   ,
       -0.21656999,  0.19023776, -3.84866   , -0.257967  ,  0.21277945,
        0.00940701,  0.19321555, -0.32837075, -0.29023   , -0.09518776,
       -0.1398064 , -0.06628689, -0.53203714,  0.3393022 ,  0.14804935],
      dtype=float32)

Визуализируем лишь небольшое подмножество фраз:

In [ ]:
from sklearn.cluster import DBSCAN, KMeans

In [ ]:
kmeans = KMeans(n_clusters=9)


In [ ]:
labels = kmeans.fit_predict(phrase_vectors)

In [ ]:
from collections import Counter

In [ ]:
Counter(labels)

Counter({8: 143, 6: 131, 0: 97, 4: 116, 3: 131, 2: 116, 1: 92, 5: 106, 7: 69})

In [ ]:
color_map = ['red', 'blue', 'green', 'yellow', 'orange', 'magenta', 'cyan', 'black', 'purple', ]

In [ ]:
colors = [color_map[i] for i in labels]

In [ ]:
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases and turn them to numpy array
phrase_vectors = np.asarray([get_phrase_embedding(x) for x in chosen_phrases])
phrase_vectors_2d = umap.UMAP(n_neighbors=3).fit_transform(phrase_vectors)

In [ ]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1], color=colors,
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

Figure(id='1366', ...)

Дополнительно, вы можете попробовать сделать следующее:
* Воспользоваться t-SNE вместо UMAP (требует гораздо больше времени на больших выборках)
* Визуализировать весь набор данных, а не только его часть
* Воспользоваться другими эмбеддингами из `gensim` "model zoo": `gensim.downloader.info()`
* Рассмотреть принципы работы [FastText](https://github.com/facebookresearch/fastText), которые несколько отличаются от word2vec

#### Выводы:
* Использование векторных представлений слов, обученных на значительных объемах текстовых данных, позволяет переносить эти знания в другие задачи.
* При построении некоторого решения для работы с текстами полезно использовать предобученные эмбеддинги. Конечно, их можно и дообучить под задачу.
* Построение эмбеддингов возможно не только для слов, но и для других объектов (текстов, изображений, графов и др.).